## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Elizabeth City,US,36.29,-76.25,62.60,72,20,5.82,few clouds
1,1,Novo Oriente,BR,-5.53,-40.77,82.27,47,59,3.13,broken clouds
2,2,Timmins,CA,48.47,-81.33,32.00,98,75,2.24,broken clouds
3,3,New Norfolk,AU,-42.78,147.06,64.00,53,79,1.99,broken clouds
4,4,Punta Arenas,CL,-53.15,-70.92,50.00,53,40,26.40,scattered clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [16]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Novo Oriente,BR,-5.53,-40.77,82.27,47,59,3.13,broken clouds
11,11,Siparia,TT,10.13,-61.50,79.00,83,75,8.05,shower rain
16,16,Molina,CL,-35.12,-71.28,77.00,31,0,14.99,clear sky
20,20,Atuona,PF,-9.80,-139.03,78.12,75,54,17.74,broken clouds
24,24,Avarua,CK,-21.21,-159.78,78.80,61,82,8.05,broken clouds
25,25,Cabo San Lucas,MX,22.89,-109.91,82.40,51,20,8.05,few clouds
27,27,Praia,CV,14.92,-23.51,78.80,74,75,16.11,haze
34,34,San Blas,MX,21.52,-105.27,86.00,42,5,9.17,clear sky
36,36,Bambous Virieux,MU,-20.34,57.76,75.20,94,75,14.99,shower rain
37,37,Tual,ID,-5.67,132.75,84.38,69,56,4.14,broken clouds


In [17]:
# 4a. Determine if there are any empty rows.
# clean_df.isnull().sum()
clean_df.count()

Unnamed: 0             190
City                   190
Country                189
Lat                    190
Lng                    190
Max Temp               190
Humidity               190
Cloudiness             190
Wind Speed             190
Current Description    190
dtype: int64

In [18]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# clean_df2 = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
#                                        (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df2 = clean_df.dropna()
clean_df2.count()
# clean_df2.isnull().sum()

Unnamed: 0             189
City                   189
Country                189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [19]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df2[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Novo Oriente,BR,82.27,broken clouds,-5.53,-40.77,
11,Siparia,TT,79.00,shower rain,10.13,-61.50,
16,Molina,CL,77.00,clear sky,-35.12,-71.28,
20,Atuona,PF,78.12,broken clouds,-9.80,-139.03,
24,Avarua,CK,78.80,broken clouds,-21.21,-159.78,
25,Cabo San Lucas,MX,82.40,few clouds,22.89,-109.91,
27,Praia,CV,78.80,haze,14.92,-23.51,
34,San Blas,MX,86.00,clear sky,21.52,-105.27,
36,Bambous Virieux,MU,75.20,shower rain,-20.34,57.76,
37,Tual,ID,84.38,broken clouds,-5.67,132.75,


In [22]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):

        print("Hotel not found... skipping.")

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
hotel_df.tail(20)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
623,Charters Towers,AU,81.21,scattered clouds,-20.10,146.27,Cattleman's Rest Motor Inn
625,Kambia,SL,77.22,scattered clouds,9.12,-12.92,Hotel Moriya
626,Mount Isa,AU,86.00,clear sky,-20.73,139.50,ibis Styles Mt Isa Verona
631,Salta,AR,87.80,broken clouds,-24.79,-65.41,
634,Piacabucu,BR,76.89,clear sky,-10.41,-36.43,Piaçabuçu News
635,Kulhudhuffushi,MV,81.81,scattered clouds,6.62,73.07,Haajy Guest House 1
636,Port Blair,IN,80.67,clear sky,11.67,92.75,
637,Noumea,NC,86.00,clear sky,-22.28,166.46,Hôtel Le Lagon
661,Lazaro Cardenas,MX,82.53,clear sky,17.96,-102.20,Collection O Hotel Quinta Antigua
665,Boyolangu,ID,77.25,broken clouds,-8.12,111.89,Yos ega cv


In [24]:
# 7. Drop the rows where there is no Hotel Name.
# "" & NAs
new_df = hotel_df[hotel_df['Hotel Name']!= ""]
new_df


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Novo Oriente,BR,82.27,broken clouds,-5.53,-40.77,Pousada e Restaurante Sol Nascente
11,Siparia,TT,79.00,shower rain,10.13,-61.50,Brook Gardens
16,Molina,CL,77.00,clear sky,-35.12,-71.28,Hostal Pucará
20,Atuona,PF,78.12,broken clouds,-9.80,-139.03,Villa Enata
24,Avarua,CK,78.80,broken clouds,-21.21,-159.78,Paradise Inn
...,...,...,...,...,...,...,...
673,Itarema,BR,80.42,overcast clouds,-2.92,-39.92,Windvillage
681,Road Town,VG,78.80,few clouds,18.42,-64.62,"ZINGARA 76ft Catamaran, full crew-all included"
686,Saint-Georges,GD,78.80,broken clouds,12.06,-61.75,Siesta Hotel
689,Nanakuli,US,86.00,scattered clouds,21.39,-158.15,Camp Pālehua


In [25]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))